In [1]:
from scripts.StreamProcessing import Stream_Data, ClimateChangeDataset # found in the scripts folder inside StreamProcessing.py. This class uses PySpark

PySpark found


https://berkeleyearth.org/data/

### A note on block 

These might take a while as PySpark attempts to download some additional files

In [2]:
# Initialize the Stream_Data Class using the topic name/s established in the kafka-config/kafka-config.ipynb and add the host of your kafka instance in the host parameter
emissions_config = Stream_Data(topics='ghg_data', host='localhost:9092')
e_session, query, emissions_df = emissions_config.getData()
#temperatures

/media/camagakhan/DATA/Repositories/BigDataProcessing/Assignment/BigDataProcessingClimateChange/notebooks/scripts/mysql-connector-j-8.0.33.jar
/media/camagakhan/DATA/Repositories/BigDataProcessing/Assignment/BigDataProcessingClimateChange/notebooks/scripts/graphframes-0.8.2-spark3.2-s_2.12.jar


23/05/16 23:20:03 WARN Utils: Your hostname, camagakhan-Inspiron-7577 resolves to a loopback address: 127.0.1.1; using 192.168.6.71 instead (on interface wlp60s0)
23/05/16 23:20:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/camagakhan/spark-3.4.0-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/camagakhan/.ivy2/cache
The jars for the packages stored in: /home/camagakhan/.ivy2/jars
org.apache.spark#spark-streaming-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6fdcdd4e-26c8-49cf-a58f-fbbf0ea7650d;1.0
	confs: [default]
	found org.apache.spark#spark-streaming-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#com

In [3]:
# Initialize the Stream_Data Class using the topic name/s established in the kafka-config/kafka-config.ipynb and add the host of your kafka instance in the host parameter
temperature_config = Stream_Data(topics='temperature', host='localhost:9092')
t_session, query_temperature, temperature_df = temperature_config.getData()

/media/camagakhan/DATA/Repositories/BigDataProcessing/Assignment/BigDataProcessingClimateChange/notebooks/scripts/mysql-connector-j-8.0.33.jar
/media/camagakhan/DATA/Repositories/BigDataProcessing/Assignment/BigDataProcessingClimateChange/notebooks/scripts/graphframes-0.8.2-spark3.2-s_2.12.jar


### NOTE
Everything is configered while initializing the PySpark Session in the ```StreamProcessing.py``` class. DO NOT remove the ```from graphframes import GraphFrame```, as the jar files are loaded while getting the data from the previous code block

In [4]:
from graphframes import GraphFrame
import findspark as tt
tt.init(emissions_config.getSpark_Path()) 
from pyspark.sql import functions as F

In [5]:
print(temperature_df)
print(emissions_df)

DataFrame[REF_AREA: string, Measure: string, UNIT_MEASURE: string, TIME_PERIOD: int, OBS_VALUE: double, REF_CODE: string]
DataFrame[PREFIX: string, Country: string, POL: string, Pollutant: string, VAR: string, Variable: string, YEA: int, Year: int, UnitCode: string, Unit: string, PowerCodeCode: string, PowerCode: string, ReferencePeriodCode: float, ReferencePeriod: float, Value: double, FlagCodes: string, Flags: string]


In [6]:
temp_df = temperature_df.withColumnRenamed('REF_CODE','COUNTRY_CODE')
ghg_df = emissions_df.withColumnRenamed('PREFIX', 'COUNTRY_CODE')

myDataset = temp_df.join(ghg_df, (temp_df.COUNTRY_CODE == ghg_df.COUNTRY_CODE) & (temp_df.TIME_PERIOD == ghg_df.Year), 'inner')


myDataset = myDataset.select(temp_df.COUNTRY_CODE, ghg_df.Country,ghg_df.Year, temp_df.UNIT_MEASURE, temp_df.OBS_VALUE, ghg_df.POL, ghg_df.VAR, ghg_df.Value)

# ran into out of memory exception

# We'll be assigning the country_codes as ids to map relationships

Store the Dataset before we attempt to use models

In [7]:
config_train = Stream_Data(topics=None, host='localhost:9092')
config_train.storeData('climate_change', myDataset) # we are about to store the new dataset in a mySQL table

Machine Learning Section.

In [8]:
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline

In the following code block we are setting up the Linear Regression to set up a model to predict the temperatures based on the given Green house Gases levels

In [9]:
assembler = VectorAssembler(inputCols=['Value'], outputCol='features')
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures')
linear_regression = LinearRegression(labelCol='OBS_VALUE', featuresCol='scaledFeatures')
'''
    Stages

    1) Get the features of the model,
    2) Normalise the features using Standard Scaler
    3) apply training using linear regression. This uses the OBS_VALUE (which is the temperature value) as Y
'''
pipeline = Pipeline(stages=[assembler, scaler, linear_regression])

In [12]:
climate_change_df = ClimateChangeDataset().loadDataset()
climate_change_df.show()

/media/camagakhan/DATA/Repositories/BigDataProcessing/Assignment/BigDataProcessingClimateChange/notebooks/scripts/graphframes-0.8.2-spark3.2-s_2.12.jar
+------------+--------------+----+------------------+---------+---+----------+----------+
|COUNTRY_CODE|       Country|Year|      UNIT_MEASURE|OBS_VALUE|POL|       VAR|     Value|
+------------+--------------+----+------------------+---------+---+----------+----------+
|         AUT|       Austria|2007|C: Degrees celsius|     1.19|GHG|     TOTAL| 86841.362|
|         AUT|       Austria|2007|C: Degrees celsius|     1.19|GHG|INDEX_1990|   110.734|
|         AUT|       Austria|2007|C: Degrees celsius|     1.19|GHG|   GHG_CAP|    10.469|
|         GBR|United Kingdom|2000|C: Degrees celsius|     0.25|GHG|     TOTAL|714300.902|
|         GBR|United Kingdom|2000|C: Degrees celsius|     0.25|GHG|INDEX_1990|    89.622|
|         GBR|United Kingdom|2000|C: Degrees celsius|     0.25|GHG|   GHG_CAP|     12.13|
|         EST|       Estonia|2015|C: D

In [13]:
train, test = climate_change_df.randomSplit([0.8, 0.2], seed=42)

In [14]:
train.show()

+------------+---------+----+------------------+---------+---+------------+---------+
|COUNTRY_CODE|  Country|Year|      UNIT_MEASURE|OBS_VALUE|POL|         VAR|    Value|
+------------+---------+----+------------------+---------+---+------------+---------+
|         ARG|Argentina|1990|C: Degrees celsius|     0.03|GHG|         AGR|105060.83|
|         ARG|Argentina|1990|C: Degrees celsius|     0.03|GHG|       AGR_P|   47.703|
|         ARG|Argentina|1990|C: Degrees celsius|     0.03|GHG|      ENER_P|   44.372|
|         ARG|Argentina|1990|C: Degrees celsius|     0.03|GHG|     GHG_CAP|     6.76|
|         ARG|Argentina|1990|C: Degrees celsius|     0.03|GHG|GHG_CAP_LULU|    8.837|
|         ARG|Argentina|1990|C: Degrees celsius|     0.03|GHG|GHG_GDP_LULU|    0.646|
|         ARG|Argentina|1990|C: Degrees celsius|     0.03|GHG|  INDEX_2000|   78.571|
|         ARG|Argentina|1990|C: Degrees celsius|     0.03|GHG|    IND_PROC|  9540.84|
|         ARG|Argentina|1990|C: Degrees celsius|     0

In [16]:
val, test = test.randomSplit([.5, .5], seed=42)

In [17]:
val.show()

+------------+---------+----+------------------+---------+---+------------+---------+
|COUNTRY_CODE|  Country|Year|      UNIT_MEASURE|OBS_VALUE|POL|         VAR|    Value|
+------------+---------+----+------------------+---------+---+------------+---------+
|         ARG|Argentina|1990|C: Degrees celsius|     0.03|GHG|       TOTAL|220240.77|
|         ARG|Argentina|1992|C: Degrees celsius|    -0.73|GHG|       AGR_P|   45.756|
|         ARG|Argentina|1992|C: Degrees celsius|    -0.73|GHG|       TOTAL|234122.38|
|         ARG|Argentina|1993|C: Degrees celsius|    -0.23|GHG|         AGR|109493.54|
|         ARG|Argentina|1994|C: Degrees celsius|     0.35|GHG|  INDEX_2000|   86.904|
|         ARG|Argentina|1996|C: Degrees celsius|     0.05|GHG|    IND_PROC|  9836.97|
|         ARG|Argentina|1996|C: Degrees celsius|     0.05|GHG|         WAS|  9834.46|
|         ARG|Argentina|1999|C: Degrees celsius|    -0.15|GHG|GHG_GDP_LULU|    0.523|
|         ARG|Argentina|1999|C: Degrees celsius|    -0

In [18]:
test.show()

+------------+---------+----+------------------+---------+---+----------+---------+
|COUNTRY_CODE|  Country|Year|      UNIT_MEASURE|OBS_VALUE|POL|       VAR|    Value|
+------------+---------+----+------------------+---------+---+----------+---------+
|         ARG|Argentina|1990|C: Degrees celsius|     0.03|GHG|      ENER| 97725.32|
|         ARG|Argentina|1990|C: Degrees celsius|     0.03|GHG|   GHG_GDP|    0.494|
|         ARG|Argentina|1990|C: Degrees celsius|     0.03|GHG|INDEX_1990|    100.0|
|         ARG|Argentina|1991|C: Degrees celsius|    -0.28|GHG|      ENER|107550.28|
|         ARG|Argentina|1991|C: Degrees celsius|    -0.28|GHG|   GHG_GDP|     0.53|
|         ARG|Argentina|1991|C: Degrees celsius|    -0.28|GHG|    LULUCF| 68967.49|
|         ARG|Argentina|1992|C: Degrees celsius|    -0.73|GHG|IND_PROC_P|    3.547|
|         ARG|Argentina|1992|C: Degrees celsius|    -0.73|GHG|    LULUCF|  70244.6|
|         ARG|Argentina|1992|C: Degrees celsius|    -0.73|GHG|       WAS|  8